In [1]:
# prepare packages
import requests, pandas as pd
from datetime import datetime, timedelta, timezone
from bs4 import BeautifulSoup
import logging
# from pythonjsonlogger import jsonlogger
import logmatic
import os

import sys
import logstash
import time, random
import pymongo
# 現狀 抓2~10頁 並塞入DB

In [14]:
# 建立 MongoClient
host = '192.168.114.130'
client = "mongodb://" + host + ":27017/"
# 多執行續下 可能被 dead lock 
myclient = pymongo.MongoClient(client, connect=False)
mydb = myclient["mydatabase"]
mycol = mydb["Ptt_fix2"]




# insert many to mongo
# x = mycol.insert_many(mylist)
# many should be a list of dict

# 傳回 _id 的值
# print(x.inserted_ids)

In [3]:
# memory
#url = "https://www.ptt.cc/bbs/Food/index2.html"

# set proxy to use
proxylist = [{"http": "37.187.120.123:80"},
             {"https": "206.189.36.198:8080"},
             {"https": "178.128.31.153:8080"},
             {"http": "167.114.180.102:8080"},
             {"http": "104.131.214.218:80"},
             {"http": "167.114.196.153:80"}]
header = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'}


In [4]:
# 直接傳進 Logstash 在進入 Elasticsearch


logger = logging.getLogger('python-logstash-logger')
logger.setLevel(logging.INFO)
host_demo = "logstash"
logger.addHandler(logstash.TCPLogstashHandler(host_demo,5959))





In [5]:
# 製作 url List
def get_url(n=2,m=3):
    result = []
    # 增加直覺 從第n頁 到 第m頁
    m = m + 1
    for t in range(n,m):
        url = "https://www.ptt.cc/bbs/Food/index" + str(t) + ".html"
        result.append(url)
    return result

In [ ]:
#new_result
get_connect("https://www.ptt.cc/bbs/Food/index.html")

In [11]:
# slove proxy connect error try next
def get_connect(url):
    i = 0
    
    while True :
        
        try :
            resp= requests.get(url,headers=header,proxies=proxylist[i])          
            resp.encoding = "utf-8"
            time.sleep(random.randint(1,4))
           
            
        except requests.exceptions.ConnectionError :             
            logger.warning("Proxy Error",extra={"proxy":proxylist[i]})
            
            i = (i + 1) % 6
            continue

            
            
        if resp : 
            if resp.status_code == 200 :
                code = "HTTP response code = " + str(resp.status_code)
                logger.info(code,extra={"url" : url, "time_use" : resp.elapsed.total_seconds(), "proxy_use":proxylist[i]})
                
                break         
        
        
    return resp.text 

In [12]:
def parse(urls):
    # 進入ptt列表 抓取 標題、推文數、網址
    result = []
    try:
        soup = BeautifulSoup(get_connect(urls))
    except Exception : 
        logger.error("index error",extra={"url":urls})
    if soup :    
        print("進入ptt : ", urls)
        post = soup.find_all("div", class_="r-ent")
        global href,title,author,date_in_content,content
        pos = []
        neg = []
        arrow = []
        score = 0
        push = 0
        for r in post:

            if r.find("div", class_="nrec").text:
                try:
                    push = int(r.find("div", class_="nrec").text)
                except ValueError:
                    pass
            if r.find("a"):
                # 取得網址
                href = "http://www.ptt.cc" + r.find("a")["href"]
                # 取得標題
                title = r.find("a").text

                # 進入第二層 進入每一篇
                # 去除公告 請益 為分類的文章
                if "公告" not in title:
                    if  "請益" not in title:
                        print("現在處理: ", href)
                        try:
                            art = BeautifulSoup(get_connect(href))
                        except Exception :
                            logger.error("href error",extra={"href":href})
                            continue
                        time.sleep(random.randint(1, 4))

                        content = art.find("div", id="main-content")
                        # 保存要丟棄的資訊
                        val = content.find_all("span", {"class": "article-meta-value"})
                        #  文章排版問題 有些文章沒有部份以下資訊
                        try:
                            author = val[0].text

                            date_in_content = val[3].text
                        except IndexError as e:
                            logger.warning("article span list out of index", extra={"url": href})
                            pass

                        # 開始丟棄資訊
                        removes = content.find_all("div", class_="article-metaline")
                        for remove in removes:
                            remove.extract()
                        removes = content.find_all("div", class_="article-metaline-right")
                        for remove in removes:
                            remove.extract()
                        removes = content.find_all("span", class_="f2")
                        for remove in removes:
                            if "※" in remove.text:
                                remove.extract()
                        # 處理推噓文
                        ps = content.find_all("div", class_="push")
                        for p in ps:
                            tag = p.find("span", class_="push-tag").text
                            if "推" in tag:
                                score = score + 1
                                # replace 第三個參數 只置換第一個
                                # push_content = p.find("span", class_="push-content").text.replace(": ","",1)
                                push_content = p.find("span", class_="push-content").text
                                push_ID = p.find("span", class_="push-userid").text
                                push_total = push_ID + push_content
                                pos.append(push_total)
                            elif "噓" in tag:
                                score = score - 1
                                push_content = p.find("span", class_="push-content").text
                                push_ID = p.find("span", class_="push-userid").text
                                push_total = push_ID + push_content
                                neg.append(push_total)
                            else:
                                push_content = p.find("span", class_="push-content").text
                                push_ID = p.find("span", class_="push-userid").text
                                push_total = push_ID + push_content
                                arrow.append(push_total)
                            p.extract()


                        try:
                            content = content.text.replace("\r", "").replace("\n", "").replace(" ","")
                        except AttributeError as e:
                            logger.warning("content is wrong in this page", extra={"url": href})
                            pass
                        result.append({
                            "title": title,
                            "author": author,
                            "date": date_in_content,
                            "push": push,
                            "href": href,
                            "content": content,
                            "score": score,
                            "推文": pos,
                            "噓文": neg,
                            "箭頭": arrow

                        })
        try:
            x = mycol.insert_many(result)
            if x.inserted_ids :
                logger.info("Insert Successful",extra={"count":len(x.inserted_ids)})
        except Exception as e:
            logger.warning("DB error",extra={"error_name":e, "index_url" : urls})



    return result

In [ ]:
from concurrent.futures import ThreadPoolExecutor, wait, ProcessPoolExecutor, as_completed

new_result = []
with ProcessPoolExecutor(max_workers= 10) as executor:
    future_result = {executor.submit(parse, url) : url for url in get_url(504,1003)}

    

    for future in as_completed(future_result):
        
        print("work has done")
        
    
    

進入ptt :  https://www.ptt.cc/bbs/Food/index513.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1342110160.A.C23.html
進入ptt :  https://www.ptt.cc/bbs/Food/index505.html
進入ptt :  https://www.ptt.cc/bbs/Food/index509.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1342015751.A.0A6.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1341929154.A.6E8.html
進入ptt :  https://www.ptt.cc/bbs/Food/index508.html
進入ptt :  https://www.ptt.cc/bbs/Food/index512.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1341997460.A.F9C.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1342097287.A.B55.html
進入ptt :  https://www.ptt.cc/bbs/Food/index504.html
進入ptt :  https://www.ptt.cc/bbs/Food/index507.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1341900214.A.1C7.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1341976263.A.966.html
進入ptt :  https://www.ptt.cc/bbs/Food/index511.html
進入ptt :  https://www.ptt.cc/bbs/Food/index506.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1342070978.A.67C.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1341934202.A.0A4.html
進入ptt :  https://ww

現在處理:  http://www.ptt.cc/bbs/Food/M.1342105741.A.5A3.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1341923597.A.BE1.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1341986858.A.575.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1342025850.A.4F0.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1342067532.A.FDE.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1341972042.A.170.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1342142409.A.339.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1342106118.A.506.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1342089464.A.32D.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1341932845.A.133.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1341925068.A.4B6.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1342027552.A.045.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1342010212.A.BBD.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1341992356.A.B01.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1342068081.A.FB9.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1342106152.A.6AE.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1342095439.A.94D.html
現在處理:  http://

現在處理:  http://www.ptt.cc/bbs/Food/M.1342365817.A.EA2.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1342324437.A.F16.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1342193199.A.61D.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1342164556.A.AC5.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1342359131.A.411.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1342411532.A.0ED.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1342234483.A.5B4.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1342278723.A.C07.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1342327132.A.C45.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1342359529.A.CB6.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1342440413.A.341.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1342193957.A.84D.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1342164858.A.544.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1342412697.A.5AB.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1342278937.A.28A.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1342367230.A.922.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1342235150.A.ECB.html
現在處理:  http://

現在處理:  http://www.ptt.cc/bbs/Food/M.1342604296.A.42C.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1342312433.A.3AE.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1342432451.A.21A.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1342450366.A.0CA.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1342503008.A.78C.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1342539717.A.E3C.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1342454998.A.C1A.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1342524619.A.14F.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1342433261.A.CF5.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1342539807.A.222.html
work has done
現在處理:  http://www.ptt.cc/bbs/Food/M.1342604416.A.809.html
進入ptt :  https://www.ptt.cc/bbs/Food/index531.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1342647200.A.1DE.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1342526513.A.43B.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1342455175.A.395.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1342450644.A.A80.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1342605123.A.819.html
現在處理:  

現在處理:  http://www.ptt.cc/bbs/Food/M.1342573080.A.F04.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1342698576.A.75A.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1342536782.A.F22.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1342681860.A.4ED.html
work has done
work has done
現在處理:  http://www.ptt.cc/bbs/Food/M.1342619763.A.413.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1342631046.A.005.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1342596658.A.5C3.html
進入ptt :  https://www.ptt.cc/bbs/Food/index535.html
進入ptt :  https://www.ptt.cc/bbs/Food/index534.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1342759716.A.418.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1342720079.A.5E7.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1342699999.A.E5C.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1342713809.A.776.html
work has done
現在處理:  http://www.ptt.cc/bbs/Food/M.1342573316.A.5C0.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1342632725.A.E01.html
進入ptt :  https://www.ptt.cc/bbs/Food/index536.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1342792205.A.C7

...

現在處理:  http://www.ptt.cc/bbs/Food/M.1345187283.A.CCB.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1345121955.A.80F.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1345286130.A.8C7.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1345258546.A.039.html
進入ptt :  https://www.ptt.cc/bbs/Food/index628.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1345379513.A.C92.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1345205469.A.E14.html
進入ptt :  https://www.ptt.cc/bbs/Food/index626.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1345304861.A.113.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1345287673.A.4FB.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1345217172.A.EF3.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1345356268.A.0D4.html
work has done
現在處理:  http://www.ptt.cc/bbs/Food/M.1345380348.A.4C4.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1345187494.A.689.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1345258863.A.DAD.html
進入ptt :  https://www.ptt.cc/bbs/Food/index629.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1345397239.A.168.html
現在處理:  http://www.ptt

現在處理:  http://www.ptt.cc/bbs/Food/M.1345433167.A.937.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1345342128.A.7B3.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1345473079.A.533.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1345298525.A.7D5.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1345534175.A.6BC.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1345473255.A.7BE.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1345375909.A.D8D.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1345299921.A.764.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1345434462.A.103.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1345343517.A.B7A.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1345454239.A.0E7.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1345473664.A.112.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1345437583.A.CB3.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1345302060.A.4C5.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1345535147.A.312.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1345376480.A.2B7.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1345454861.A.D9E.html
現在處理:  http://

現在處理:  http://www.ptt.cc/bbs/Food/M.1345555324.A.D7A.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1345564530.A.A05.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1345691959.A.F48.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1345543201.A.975.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1345643788.A.657.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1345525295.A.D5F.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1345638257.A.800.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1345707836.A.F27.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1345645547.A.5E7.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1345555826.A.AEB.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1345565093.A.166.html
work has done
現在處理:  http://www.ptt.cc/bbs/Food/M.1345692387.A.6E4.html
進入ptt :  https://www.ptt.cc/bbs/Food/index643.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1345742053.A.FDA.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1345613724.A.98E.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1345711958.A.7ED.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1345565236.A.9A9.html
現在處理:  

進入ptt :  https://www.ptt.cc/bbs/Food/index648.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1345906477.A.A80.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1345737327.A.9CE.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1345868910.A.8A2.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1345919309.A.5C5.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1345794827.A.A36.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1345776293.A.C5E.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1345816179.A.528.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1345726537.A.7FB.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1345893696.A.5E3.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1345906514.A.2AF.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1345870521.A.106.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1345737421.A.3FB.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1345816706.A.B8D.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1345899234.A.D1E.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1345776851.A.565.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1345795330.A.8F4.html
現在處理:  http://www.ptt

現在處理:  http://www.ptt.cc/bbs/Food/M.1345985808.A.9AD.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1346081664.A.2BE.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1346075608.A.AA7.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1345955490.A.88A.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1345997995.A.210.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1345914468.A.855.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1346122807.A.F5E.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1345990314.A.9E5.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1346082277.A.703.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1346075663.A.33B.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1345917391.A.449.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1345998067.A.B79.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1346059206.A.AF5.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1345956981.A.230.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1346082376.A.494.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1345999900.A.CA3.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1345990360.A.0A2.html
現在處理:  http://

In [ ]:
import pandas as pd
df = pd.DataFrame(columns=["title","author","date","push","href","content","score","推文","噓文","箭頭"])
for r in new_result:
    for evey in r:
        s = pd.Series([evey["title"],evey["author"],evey["date"],evey["push"],evey["href"],evey["content"],evey["score"],evey["推文"],evey["噓文"],evey["箭頭"]]
                      ,index=["title","author","date","push","href","content","score","推文","噓文","箭頭"])
        df = df.append(s,ignore_index=True)
        

In [ ]:
df



In [ ]:
import json

collect = json.loads(df.T.to_json()).values()

In [ ]:
from pymongo import MongoClient

class MongoBase:
    def __init__(self,collection):
        self.collection=collection
        self.OpenDB()
    def OpenDB(self):
        user='root'
        passwd='root'
        # on local host
        host='192.168.114.130'
        port='27017'
        auth_db='admin'
        uri = "mongodb://"+user+":"+passwd+"@"+host+":"+port+"/"+auth_db+"?authMechanism=SCRAM-SHA-1"
        self.con = MongoClient(uri, connect=True)
        self.db=self.con['wangdong']
        self.collection=self.db[self.collection]
    def closeDB(self):
        self.con.close()
        
mongo = MongoBase('test3')

In [ ]:


mongo.collection.insert_many(collect)
mongo.closeDB()


In [ ]:
parse("https://www.ptt.cc/bbs/Food/index7004.html")

In [ ]:
# 進入ptt列表 抓取 標題、推文數、網址 
result = []
soup = BeautifulSoup(get_connect(url))
print("進入ptt : ",url)
post = soup.find_all("div", class_="r-ent")
for r in post:
            # 取得推的數量
    push = 0
    if r.find("div", class_="nrec").text:
        try:
            push = int(r.find("div", class_="nrec").text)
        except ValueError:
                pass
    if r.find("a"):
        # 取得網址
        href = "http://www.ptt.cc" + r.find("a")["href"]
        # 取得標題
        title = r.find("a").text
        
        # 進入第二層 進入每一篇
        # 去除公告 請益 為分類的文章
        if not "公告" in title :
            if not "請益" in title:
                print("現在處理: ", href)
                art = BeautifulSoup(get_connect(href))
                article = []
                content = art.find("div", id="main-content")
                # 保存要丟棄的資訊
                val = content.find_all("span", {"class":"article-meta-value"})
                author = val[0].text
                title_in_content = val[2].text
                date_in_content = val[3].text
                
                # 開始丟棄資訊
                removes = content.find_all("div", class_="article-metaline")
                for remove in removes:
                    remove.extract()
                removes = content.find_all("div", class_="article-metaline-right")
                for remove in removes:
                    remove.extract()
                removes = content.find_all("span", class_="f2")
                for remove in removes:
                    remove.extract()
                removes = content.find_all("div", class_="push")
                for remove in removes:
                    remove.extract()
                article.append({
                    "author" : author,
                    "title_in_content" : title_in_content,
                    "date_in_content" : date_in_content,
                    "content" : content.text
                })
                
    result.append({
        "title" : title,
        "push" : push,
        "href" : href,
        "content" : article
        
    })
    
    time.sleep(random.randint(1,6))
                






In [ ]:
import json
# 轉成 Dataframe
df = pd.DataFrame(result)



In [ ]:
# 排列欄位順序

df = df[ ["title", "push", "href", "content" ] ]

In [ ]:
df.iloc[1]

In [ ]:
# I/O 較多 使用 thread
start = time.time()
with ThreadPoolExecutor(max_workers=4) as executor:
    results = executor.map(int, ['1', '2', '3', '4', '5'])
    for v in results:
        print(v)
end = time.time()
print("time used :",end - start)